In [1]:
import pandas as pd
from neo4j import GraphDatabase
from neo4j.exceptions import AuthError
import json
from tqdm import tqdm

In [3]:
# df_ireland = pd.read_csv('../thesis_modularized/data/gpt_gemini_ireland_2iter_summ.csv')
# df_poland = pd.read_csv('../thesis_modularized/data/gpt_gemini_poland_2iter_summ.csv')
# df = pd.concat([df_ireland, df_poland], ignore_index=True)

df = pd.read_csv('../thesis_modularized/data/pdf_processing_results.csv')
df

,pdf_name,model_type,iteration_number,extraction_output,refined_output
0,IE-268.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""Accident ...","{\n ""nodes"": [\n {\n ""id"": ""Accident ..."
1,IE-0282.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""Collision...","{\n ""nodes"": [\n {\n ""id"": ""Collision..."
2,IE-415.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""08011001""...","{\n ""nodes"": [\n {\n ""id"": ""08011001""..."
3,IE-433.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""RAIU Repo...","{\n ""nodes"": [\n {\n ""id"": ""RAIU Repo..."
4,IE-487.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""RAIU 3 Re...","{\n ""nodes"": [\n {\n ""id"": ""RAIU 3 Re..."
...,...,...,...,...,...
57,PL-10018.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""Accident ...","{\n ""nodes"": [\n {\n ""id"": ""Accident ..."
58,PL-10022.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""Gogolin S...","{\n ""nodes"": [\n {\n ""id"": ""Gogolin S..."
59,PL-10030.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""Grodzisko...","{\n ""nodes"": [\n {\n ""id"": ""Grodzisko..."
60,PL-10041.pdf,gemini-1.5-flash-latest,1,"{\n ""nodes"": [\n {\n ""id"": ""Train acc...","{\n ""nodes"": [\n {\n ""id"": ""Train acc..."


In [4]:
# Neo4j Connection Setup
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

try:
    # Test the connection
    with driver.session() as session:
        session.run("RETURN 1")
    print("Connected to Neo4j successfully.")
except AuthError as e:
    print("Authentication failed. Check your credentials:", str(e))

Connected to Neo4j successfully.


In [6]:
def push_to_neo4j(df, model, iteration, use_refined=True):

    df = df[(df["model_type"] == model) & (df["iteration_number"] == iteration)]
    
    column = "refined_output" if use_refined else "extraction_output"
    
    for _, row in df.iterrows():
        json_str = row.get(column)
        if not json_str:
            continue

        try:
            data = json.loads(json_str)
        except json.JSONDecodeError:
            continue

        with driver.session(database=NEO4J_DATABASE) as session:
            for node in data.get("nodes", []):
                session.run(
                    f"MERGE (n:`{node.get('type', 'Unknown')}` {{id: $id}}) "
                    "ON CREATE SET n.name = $id",
                    id=node.get("id")
                )

            for rel in data.get("rels", []):
                session.run(
                    f"MATCH (a {{id: $source}}) MATCH (b {{id: $target}}) "
                    f"MERGE (a)-[r:`{rel.get('type', 'RELATED_TO')}`]->(b)",
                    source=rel.get("source"),
                    target=rel.get("target")
                )

In [7]:
push_to_neo4j(df, model="gemini-1.5-flash-latest", iteration=1, use_refined=True)

In [5]:
# Clear database
def clear_neo4j_database():
    """Delete all nodes and relationships in the Neo4j database."""
    with driver.session(database=NEO4J_DATABASE) as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("Neo4j database cleared successfully.")

# Run the function to clear the database
clear_neo4j_database()

Neo4j database cleared successfully.


In [ ]:
# Close Neo4j connection
driver.close()

In [58]:
def run_query(cypher_query):
    with driver.session() as session:
        result = session.run(cypher_query)
        return pd.DataFrame([record.data() for record in result])

In [67]:
drop_query = "CALL gds.graph.drop('thesis', false)"
run_query(drop_query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('thesis', false)"


,graphName,database,databaseLocation,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,thesis,neo4j,local,,-1,105,314,{'relationshipProjection': {'systemic_factor':...,0.028755,2025-06-14T12:14:52.391495100+02:00,2025-06-14T12:14:52.391495100+02:00,"{'graphProperties': {}, 'nodes': {'Contributin...","{'graphProperties': {}, 'nodes': {'Contributin..."


In [70]:
create_projection_query = """
CALL gds.graph.project(
  'thesis',
  ['UniqueAccident', 'AccidentType', 'ContributingFactor', 'SystemicFactor', 'Country', 'Date', 'Time', 'TrackSection', 'RegulatoryBody'],
  {
    contributing_factor: { type: 'contributing_factor', orientation: 'NATURAL' },
    systemic_factor:     { type: 'systemic_factor',     orientation: 'NATURAL' },
    occurred_in:         { type: 'occurred_in',         orientation: 'NATURAL' },
    is_type:             { type: 'is_type',             orientation: 'NATURAL' },
    is_track_section:    { type: 'is_track_section',    orientation: 'NATURAL' },
    investigated_by:     { type: 'investigated_by',     orientation: 'NATURAL' },
    has_date:            { type: 'has_date',            orientation: 'NATURAL' },
    has_time:            { type: 'has_time',            orientation: 'NATURAL' }
  }
)
"""

projection_result = run_query(create_projection_query)
projection_result

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,{'ContributingFactor': {'label': 'Contributing...,"{'systemic_factor': {'aggregation': 'DEFAULT',...",thesis,105,157,69


In [81]:
query ="""
MATCH (start_node:ContributingFactor {id: "Awareness"})-[r]-(connected)
RETURN 
  labels(connected)[0] AS connected_type,
  connected.id AS connected_id,
  type(r) AS rel_type
"""

run_query(query)

,connected_type,connected_id,rel_type
0,UniqueAccident,Chałupki - Krzyżanowice Level Crossing Acciden...,contributing_factor
1,UniqueAccident,Komorów – Podkowa Leśna Główna Accident on 19/...,contributing_factor
2,UniqueAccident,Skierniewice Accident 24/08/2023,contributing_factor


In [ ]:
query ="""
MATCH (factor:ContributingFactor {id: "Awareness"})<-[:contributing_factor]-(accident:UniqueAccident)
OPTIONAL MATCH (accident)-[:has_date]->(date:Date)
OPTIONAL MATCH (accident)-[:has_time]->(time:Time)
OPTIONAL MATCH (accident)-[:occurred_in]->(country:Country)
OPTIONAL MATCH (accident)-[:occurred_at]->(section:TrackSection)
OPTIONAL MATCH (accident)-[:is_type]->(type:AccidentType)
OPTIONAL MATCH (accident)-[:investigated_by]->(regulator:RegulatoryBody)

RETURN 
  accident.id AS accident_id,
  date.id AS date,
  time.id AS time,
  country.id AS country,
  section.id AS track_section,
  type.id AS accident_type,
  regulator.id AS regulatory_body
ORDER BY date DESC
"""

run_query(query)

,accident_id,date,time,country,track_section,accident_type,regulatory_body
0,Skierniewice Accident 24/08/2023,24/08/23,12:05,Poland,"railway line no. 1, km 64.282",Collisions,State Commission on Railway Accident Investiga...
1,Chałupki - Krzyżanowice Level Crossing Acciden...,20/02/2023,18:11,Poland,Railway line no. 151 Kędzierzyn-Koźle - Chałup...,Level Crossing Accident involving a train and ...,STATE COMMISSION ON RAILWAY ACCIDENT INVESTIGA...
2,Komorów – Podkowa Leśna Główna Accident on 19/...,19/05/23,09:51,Poland,km 18.805 of railway line no. 47 Warszawa Śród...,Level Crossing Accident,STATE COMMISSION ON RAILWAY ACCIDENT INVESTIGA...
